In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [ ]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.sqrt(np.dot(r_vec, r_vec))
    return r
    
def findCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

In [ ]:
def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

In [ ]:
def accelerationN(x_1, x_2, m_1, m_2, DL):
    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])

In [3]:
def twoOrbsNewt(s, t_max = 2000000, dt = 10):
    x_1, x_2, v_1, v_2, a_1, a_2, m_1, m_2 = s

    x_cm = findCM(x_1, x_2, m_1, m_2) 
    v_cm = findCM(v_1, v_2, m_1, m_2)

    pos1 = [], []
    pos2 = [], []
    posCM = [], []
    vel1 = [], []
    vel2 = [], []
    acc1 = [], []
    acc2 = [], []

    for i in np.arange(0, t_max, dt):
        pos1[0].append(x_1[0])
        pos1[1].append(x_1[1])
        pos2[0].append(x_2[0])
        pos2[1].append(x_2[1])
        posCM[0].append(x_cm[0])
        posCM[1].append(x_cm[1])
        vel1[0].append(v_1[0])
        vel1[1].append(v_1[1])
        vel2[0].append(v_2[0])
        vel2[1].append(v_2[1])
        acc1[0].append(a_1[0])
        acc1[1].append(a_1[1])
        acc2[0].append(a_2[0])
        acc2[1].append(a_2[1])

        a_1, a_2 = accelerationN(x_1, x_2, m_1, m_2, DL = True)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
        x_cm = findCM(x_1, x_2, m_1, m_2)

    return pos1, pos2, posCM, vel1, vel2, acc1, acc2

In [4]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def twoOrbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        for i in range(0, len(x_cm[0])):
            x_1[0][i] = x_1[0][i] - x_cm[0][i]
            x_1[1][i] = x_1[1][i] - x_cm[1][i]
            x_2[0][i] = x_2[0][i] - x_cm[0][i]
            x_2[1][i] = x_2[1][i] - x_cm[1][i]
            x_cm[0][i] = 0
            x_cm[1][i] = 0


    fig, ax = plt.subplots(figsize=(14,14))
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    ax.set_xlabel('$R_S$', fontsize = 15)
    ax.set_ylabel('$R_S$', fontsize = 15)
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Two_Body/{filename}.png', dpi=300, transparent = False)
    return


def prettyTwoOrbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        for i in range(0, len(x_cm[0])):
            x_1[0][i] = x_1[0][i] - x_cm[0][i]
            x_1[1][i] = x_1[1][i] - x_cm[1][i]
            x_2[0][i] = x_2[0][i] - x_cm[0][i]
            x_2[1][i] = x_2[1][i] - x_cm[1][i]
            x_cm[0][i] = 0
            x_cm[1][i] = 0


    fig, ax = plt.subplots(figsize=(14,14))
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    # ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    # ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    # ax.set_xlabel('$R_S$', fontsize = 15)
    # ax.set_ylabel('$R_S$', fontsize = 15)
    # ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    # ax.set_title(f'{filename}', fontsize = 20)
    # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
                
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Two_Body/Pretty/{filename}_pretty.png', dpi=300, transparent = False)
    return

In [5]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def twoOrbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        for i in range(0, len(x_cm[0])):
            x_1[0][i] = x_1[0][i] - x_cm[0][i]
            x_1[1][i] = x_1[1][i] - x_cm[1][i]
            x_2[0][i] = x_2[0][i] - x_cm[0][i]
            x_2[1][i] = x_2[1][i] - x_cm[1][i]
            x_cm[0][i] = 0
            x_cm[1][i] = 0


    fig, ax = plt.subplots(figsize=(14,14))
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    ax.set_xlabel('$R_S$', fontsize = 15)
    ax.set_ylabel('$R_S$', fontsize = 15)
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Two_Body/{filename}.png', dpi=300, transparent = False)
    return


def prettyTwoOrbPlotter(pos1, pos2, posCM, aspect = 1, filename='', CM = False, save = False, show = True):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        for i in range(0, len(x_cm[0])):
            x_1[0][i] = x_1[0][i] - x_cm[0][i]
            x_1[1][i] = x_1[1][i] - x_cm[1][i]
            x_2[0][i] = x_2[0][i] - x_cm[0][i]
            x_2[1][i] = x_2[1][i] - x_cm[1][i]
            x_cm[0][i] = 0
            x_cm[1][i] = 0


    fig, ax = plt.subplots(figsize=(14,14))
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    # ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    # ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b--', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r--', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    # ax.set_xlabel('$R_S$', fontsize = 15)
    # ax.set_ylabel('$R_S$', fontsize = 15)
    # ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    # ax.set_title(f'{filename}', fontsize = 20)
    # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
                
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Two_Body/Pretty/{filename}_pretty.png', dpi=300, transparent = False)
    return

In [7]:
def twoOrbsEIH(s, t_max = 2000000, dt = 10):
    x_1, x_2, v_1, v_2, a_1, a_2, m_1, m_2 = s

    x_cm = findCM(x_1, x_2, m_1, m_2) 
    # v_cm = findCM(v_1, v_2, m_1, m_2)

    pos1 = [], []
    pos2 = [], []
    posCM = [], []
    vel1 = [], []
    vel2 = [], []
    acc1 = [], []
    acc2 = [], []

    for i in np.arange(0, t_max, dt):
        pos1[0].append(x_1[0])
        pos1[1].append(x_1[1])
        pos2[0].append(x_2[0])
        pos2[1].append(x_2[1])
        posCM[0].append(x_cm[0])
        posCM[1].append(x_cm[1])
        vel1[0].append(v_1[0])
        vel1[1].append(v_1[1])
        vel2[0].append(v_2[0])
        vel2[1].append(v_2[1])
        acc1[0].append(a_1[0])
        acc1[1].append(a_1[1])
        acc2[0].append(a_2[0])
        acc2[1].append(a_2[1])

        a_1, a_2 = accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2)
        v_1, v_2 = boostEIH(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = moveEIH(x_1, x_2, v_1, v_2, dt)
        x_cm = findCM(x_1, x_2, m_1, m_2)

    return pos1, pos2, posCM, vel1, vel2, acc1, acc2

# ANIMATION

In [15]:
from matplotlib.animation import FuncAnimation

def twoOrbAnimator(pos1, pos2, posCM, filename, length = 10, fps = 24):
    fig = plt.figure(figsize=(12,12))
    ax = plt.subplot(1, 1, 1)
    ax.set_aspect('1')

    xlim, ylim = plotLimsTwoBody(pos1, pos2)

    def init_func():
        ax.clear()
        plt.xlabel('x')
        plt.xlabel('y')
        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)

    data_length = len(pos1[0])
    data_skip = data_length//length*fps

    def update_plot(i):
        ax.plot(pos1[0][i:i+data_skip],pos1[1][i:i+data_skip],'b',lw=1)
        ax.plot(pos2[0][i:i+data_skip],pos2[1][i:i+data_skip],'r',lw=1)
        ax.plot(posCM[0][i:i+data_skip],posCM[1][i:i+data_skip],'y:',lw=0.5)
        return ax,

    animation = FuncAnimation(fig, update_plot, frames=range(0, len(pos1[0]), data_skip), init_func=init_func, interval = 1000//fps, blit=False)
    animation.save(f'./Animations/{filename}_anim.gif', dpi=200, writer='ffmpeg')
    plt.close()